In [44]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping

from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score

import random
from sklearn.model_selection import KFold , StratifiedKFold
import optuna


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

USE_OPTUNA = False

In [31]:
import pandas as pd

submission = pd.read_csv("/kaggle/input/playground-series-s4e10/sample_submission.csv")

part_climbing_res = pd.read_csv("/kaggle/input/hill-climbing/submission.csv")

In [32]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e10/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e10/test.csv")
original = pd.read_csv("/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv")

df_train = pd.concat([df_train, original], ignore_index=True)

In [33]:
df_train = df_train.loc[df_train['person_age']<90]
df_train = df_train.loc[df_train['person_income']<400000]
df_train.loc[df_train['person_emp_length']>100, 'person_emp_length'] = 40
df_train.loc[df_train['loan_percent_income']>0.8] # Odd number. but loan was given. dont change anything


## Test few new features and if it makes the standard model more accurate
df_train['loan_income_ratio'] = df_train['loan_amnt'] / df_train['person_income']
#df_train['loan_length_ratio'] = df_train['loan_amnt'] / df_train['person_emp_length']
df_train['loan_cost'] = df_train['loan_int_rate'] * df_train['loan_amnt']
#df_train['income_to_int_ratio'] = df_train['person_income'] / df_train['loan_int_rate']
#df_train['loan_to_length_ratio'] = df_train['loan_amnt'] / df_train['cb_person_cred_hist_length']
#df_train['age_hist_ratio'] = df_train['person_age'] / df_train['cb_person_cred_hist_length']
#df_train['lenght_to_age'] = df_train['person_emp_length'] / df_train['person_age']
#df_train['loan_income_ratio'] = df_train['loan_percent_income'] * df_train['loan_amnt']
# df_train['loan_income_ratio'] = df_train['loan_amnt'] / df_train['loan_grade']



df_test['loan_income_ratio'] = df_test['loan_amnt'] / df_test['person_income']
#df_test['loan_length_ratio'] = df_test['loan_amnt'] / df_test['person_emp_length']
df_test['loan_cost'] = df_test['loan_int_rate'] * df_test['loan_amnt']
#df_test['income_to_int_ratio'] = df_test['person_income'] / df_test['loan_int_rate']
#df_test['loan_to_length_ratio'] = df_test['loan_amnt'] / df_test['cb_person_cred_hist_length']
#df_test['age_hist_ratio'] = df_test['person_age'] / df_test['cb_person_cred_hist_length']
#df_test['lenght_to_age'] = df_test['person_emp_length'] / df_test['person_age']
#df_test['loan_income_ratio'] = df_test['loan_percent_income'] * df_test['loan_amnt']

#new_features = ['loan_income_ratio', 'loan_length_ratio', 'loan_cost', 'income_to_int_ratio', 'loan_to_length_ratio', 'age_hist_ratio', 'lenght_to_age', 'loan_income_ratio']


###
# Person Home Ownership
df_train.loc[df_train['person_home_ownership'] == 'OTHER', 'person_home_ownership'] = 1
df_train.loc[df_train['person_home_ownership'] == 'RENT', 'person_home_ownership'] = 2
df_train.loc[df_train['person_home_ownership'] == 'MORTGAGE', 'person_home_ownership'] = 3
df_train.loc[df_train['person_home_ownership'] == 'OWN', 'person_home_ownership'] = 4
df_train['person_home_ownership'] = df_train['person_home_ownership'].astype(int)

df_test.loc[df_test['person_home_ownership'] == 'OTHER', 'person_home_ownership'] = 1
df_test.loc[df_test['person_home_ownership'] == 'RENT', 'person_home_ownership'] = 2
df_test.loc[df_test['person_home_ownership'] == 'MORTGAGE', 'person_home_ownership'] = 3
df_test.loc[df_test['person_home_ownership'] == 'OWN', 'person_home_ownership'] = 4
df_test['person_home_ownership'] = df_test['person_home_ownership'].astype(int)


# Loan Intent
df_train.loc[df_train['loan_intent'] == 'EDUCATION', 'loan_intent'] = 1
df_train.loc[df_train['loan_intent'] == 'MEDICAL', 'loan_intent'] = 2
df_train.loc[df_train['loan_intent'] == 'PERSONAL', 'loan_intent'] = 3
df_train.loc[df_train['loan_intent'] == 'VENTURE', 'loan_intent'] = 4
df_train.loc[df_train['loan_intent'] == 'DEBTCONSOLIDATION', 'loan_intent'] = 5
df_train.loc[df_train['loan_intent'] == 'HOMEIMPROVEMENT', 'loan_intent'] = 6
df_train['loan_intent'] = df_train['loan_intent'].astype(int)

df_test.loc[df_test['loan_intent'] == 'EDUCATION', 'loan_intent'] = 1
df_test.loc[df_test['loan_intent'] == 'MEDICAL', 'loan_intent'] = 2
df_test.loc[df_test['loan_intent'] == 'PERSONAL', 'loan_intent'] = 3
df_test.loc[df_test['loan_intent'] == 'VENTURE', 'loan_intent'] = 4
df_test.loc[df_test['loan_intent'] == 'DEBTCONSOLIDATION', 'loan_intent'] = 5
df_test.loc[df_test['loan_intent'] == 'HOMEIMPROVEMENT', 'loan_intent'] = 6
df_test['loan_intent'] = df_test['loan_intent'].astype(int)

# Loan Grade
df_train.loc[df_train['loan_grade'] == 'A', 'loan_grade'] = 1
df_train.loc[df_train['loan_grade'] == 'B', 'loan_grade'] = 2
df_train.loc[df_train['loan_grade'] == 'C', 'loan_grade'] = 3
df_train.loc[df_train['loan_grade'] == 'D', 'loan_grade'] = 4
df_train.loc[df_train['loan_grade'] == 'E', 'loan_grade'] = 5
df_train.loc[df_train['loan_grade'] == 'F', 'loan_grade'] = 6
df_train.loc[df_train['loan_grade'] == 'G', 'loan_grade'] = 7
df_train['loan_grade'] = df_train['loan_grade'].astype(int)

df_test.loc[df_test['loan_grade'] == 'A', 'loan_grade'] = 1
df_test.loc[df_test['loan_grade'] == 'B', 'loan_grade'] = 2
df_test.loc[df_test['loan_grade'] == 'C', 'loan_grade'] = 3
df_test.loc[df_test['loan_grade'] == 'D', 'loan_grade'] = 4
df_test.loc[df_test['loan_grade'] == 'E', 'loan_grade'] = 5
df_test.loc[df_test['loan_grade'] == 'F', 'loan_grade'] = 6
df_test.loc[df_test['loan_grade'] == 'G', 'loan_grade'] = 7
df_test['loan_grade'] = df_test['loan_grade'].astype(int)


# Default on File
df_train.loc[df_train['cb_person_default_on_file'] == 'N', 'cb_person_default_on_file'] = False
df_train.loc[df_train['cb_person_default_on_file'] == 'Y', 'cb_person_default_on_file'] = True
df_train['cb_person_default_on_file'] = df_train['cb_person_default_on_file'].astype(bool)

df_test.loc[df_test['cb_person_default_on_file'] == 'N', 'cb_person_default_on_file'] = False
df_test.loc[df_test['cb_person_default_on_file'] == 'Y', 'cb_person_default_on_file'] = True
df_test['cb_person_default_on_file'] = df_test['cb_person_default_on_file'].astype(bool)

df_train.drop(columns='id', inplace = True)
df_test.drop(columns='id', inplace = True)

df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91091 entries, 0 to 91225
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  91091 non-null  int64  
 1   person_income               91091 non-null  int64  
 2   person_home_ownership       91091 non-null  int64  
 3   person_emp_length           90197 non-null  float64
 4   loan_intent                 91091 non-null  int64  
 5   loan_grade                  91091 non-null  int64  
 6   loan_amnt                   91091 non-null  int64  
 7   loan_int_rate               87986 non-null  float64
 8   loan_percent_income         91091 non-null  float64
 9   cb_person_default_on_file   91091 non-null  bool   
 10  cb_person_cred_hist_length  91091 non-null  int64  
 11  loan_status                 91091 non-null  int64  
 12  loan_income_ratio           91091 non-null  float64
 13  loan_cost                   87986 no

In [34]:
cat_cols = df_train.select_dtypes(include='object').columns


df_train[cat_cols] = df_train[cat_cols].astype('category')
df_test[cat_cols] = df_test[cat_cols].astype('category')



df_train.shape

(91091, 14)

In [35]:
# negative_samples = train[train['loan_status'] == 0]
# positive_samples = train[train['loan_status'] == 1]


# negative_samples_under = negative_samples.sample(len(positive_samples) * 3, random_state=42)
# train = pd.concat([negative_samples_under, positive_samples])

# train.shape,positive_samples.shape

In [36]:
cat_cols = df_train.select_dtypes(include=['category']).columns.tolist()

onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

for col in cat_cols:
    train_encoded = onehot_encoder.fit_transform(df_train[[col]])
    df_test = onehot_encoder.transform(df_test[[col]])

    new_col_names = [f"one_hot_{i+1}_{col}" for i in range(train_encoded.shape[1])]

    df_train = pd.concat([df_train.drop(columns=[col]), pd.DataFrame(train_encoded, columns=new_col_names, index=df_train.index)], axis=1)
    df_test = pd.concat([df_test.drop(columns=[col]), pd.DataFrame(test_encoded, columns=new_col_names, index=df_test.index)], axis=1)

df_train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,loan_income_ratio,loan_cost
0,37,35000,2,0.0,1,2,6000,11.49,0.17,False,14,0,0.171429,68940.0
1,22,56000,4,6.0,2,3,4000,13.35,0.07,False,2,0,0.071429,53400.0
2,29,28800,4,8.0,3,1,6000,8.90,0.21,False,10,0,0.208333,53400.0
3,30,70000,2,14.0,4,2,12000,11.11,0.17,False,5,0,0.171429,133320.0
4,22,60000,2,2.0,2,1,6000,6.92,0.10,False,3,0,0.100000,41520.0


In [37]:
X = df_train.drop(['loan_status'], axis=1) 
y = df_train['loan_status']

In [38]:
callbacks = [log_evaluation(period=300), early_stopping(stopping_rounds=250)]
def objective(trial):
    
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'random_state': 42,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         'class_weight': {0: 1, 1: trial.suggest_uniform('class_weight_1', 1, 10)},
        'n_estimators': 3000,
        'verbose' : -1,
        'device':'gpu'
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_results = []
    
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model = lgb.LGBMClassifier(**param)
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric='logloss',
                  callbacks = callbacks
                  
                )

        y_pred = model.predict_proba(X_val)[:, 1]
        score = roc_auc_score(y_val, y_pred)
        cv_results.append(score)

    return np.mean(cv_results)

In [45]:
if USE_OPTUNA:
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)  


    print("Best hyperparameters: ", study.best_params)
    lgbm_params = study.best_params
else:

    lgbm_params = {'learning_rate': 0.02003481065310373, 'num_leaves': 28, 'max_depth': 6, 'min_data_in_leaf': 1, 'feature_fraction': 0.7163148792559001, 'bagging_fraction': 0.9426508798041959, 'bagging_freq': 5}

In [46]:
lgbm_model = lgb.LGBMClassifier(**lgbm_params)

LGBM_models = []
LGBM_cv_scores = []
oof_preds = np.zeros(len(df_train))

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(skf.split(df_train, df_train['loan_status'])):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    lgbm_model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='auc',
              callbacks = callbacks
             )
    LGBM_models.append(lgbm_model)

    y_pred = lgbm_model.predict_proba(X_val)[:, 1]
    
    oof_preds[val_index] = y_pred
    
    score = roc_auc_score(y_val, y_pred)
    LGBM_cv_scores.append(score)
    

oof_auc = roc_auc_score(df_train['loan_status'], oof_preds)
print("LGBM OOF ROC AUC: ", oof_auc)
print("LGBM CV Scores: ", LGBM_cv_scores)
print("LGBM Mean CV Score: ", np.mean(LGBM_cv_scores))

[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.7163148792559001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7163148792559001
[LightGBM] [Warning] bagging_fraction is set=0.9426508798041959, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9426508798041959
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.7163148792559001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7163148792559001
[LightGBM] [Warning] bagging_fraction is set=0.9426508798041959, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9426508798041959
[LightGBM] [Warning] bagging_freq is set=5, subsam

part_climbing_res# blending the submission by hill climbing to maximize oof CV

In [48]:
preds = np.zeros(df_test.shape[0])

for model in LGBM_models:
    preds += model.predict_proba(df_test)[:, 1] / len(LGBM_models)

part_climbing_res = pd.read_csv("/kaggle/input/playground-series-s4e10/sample_submission.csv")   
submission['loan_status'] = preds * 0.15 + part_climbing_res['loan_status'] * 0.85

submission.to_csv('submission1.csv', index=False)

submission.head()

[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.7163148792559001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7163148792559001
[LightGBM] [Warning] bagging_fraction is set=0.9426508798041959, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9426508798041959
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] feature_fraction is set=0.7163148792559001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7163148792559001
[LightGBM] [Warning] bagging_fraction is set=0.9426508798041959, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9426508798041959
[LightGBM] [Warning] bagging_freq is set=5, subsam

,id,loan_status
0,58645,0.545200
1,58646,0.434363
2,58647,0.479874
3,58648,0.433046
4,58649,0.459856


In [42]:
feature_names = df_test.columns


lgb_importances = np.zeros(len(feature_names))

for model in LGBM_models:
    lgb_importances += model.feature_importances_ / len(LGBM_models)

lgb_feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': lgb_importances})
lgb_feature_importance.sort_values(by='Importance', ascending=False, inplace=True)


print(lgb_feature_importance)

                       Feature  Importance
1                person_income       588.0
4                  loan_intent       540.0
2        person_home_ownership       350.0
11           loan_income_ratio       220.0
7                loan_int_rate       200.0
5                   loan_grade       199.0
8          loan_percent_income       178.0
6                    loan_amnt       127.0
3            person_emp_length       117.0
12                   loan_cost        85.0
0                   person_age        74.0
9    cb_person_default_on_file        13.0
10  cb_person_cred_hist_length         9.0


In [43]:
LGBM_models

[LGBMClassifier(bagging_fraction=0.9426508798041959, bagging_freq=5,
                feature_fraction=0.7163148792559001,
                learning_rate=0.02003481065310373, max_depth=6,
                min_data_in_leaf=1, num_leaves=28),
 LGBMClassifier(bagging_fraction=0.9426508798041959, bagging_freq=5,
                feature_fraction=0.7163148792559001,
                learning_rate=0.02003481065310373, max_depth=6,
                min_data_in_leaf=1, num_leaves=28),
 LGBMClassifier(bagging_fraction=0.9426508798041959, bagging_freq=5,
                feature_fraction=0.7163148792559001,
                learning_rate=0.02003481065310373, max_depth=6,
                min_data_in_leaf=1, num_leaves=28),
 LGBMClassifier(bagging_fraction=0.9426508798041959, bagging_freq=5,
                feature_fraction=0.7163148792559001,
                learning_rate=0.02003481065310373, max_depth=6,
                min_data_in_leaf=1, num_leaves=28),
 LGBMClassifier(bagging_fraction=0.9426508798041